## Paper Crawler

In [24]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import re

# Load the dataset
print("Loading AI paper dataset...")
ds = load_dataset("Seed42Lab/AI-paper-crawl")

# Focus only on NIPS, CVPR, and ICLR papers from 2010 onwards
conferences = ["NIPS", "CVPR", "ICLR"]
filtered_data = {}

for conf in conferences:
    conf_data = ds[conf]
    # Convert year to int if it's a string and filter by year ≥ 2010
    filtered_data[conf] = []
    for paper in conf_data:
        # Convert year to int if it's a string
        year = paper["year"]
        if isinstance(year, str):
            try:
                year = int(year)
            except ValueError:
                # Skip papers with invalid year format
                continue
        
        # Only include papers from 2010 onwards
        if year >= 2010:
            # Create a copy with year as int
            paper_copy = dict(paper)
            paper_copy["year"] = year
            filtered_data[conf].append(paper_copy)
    
    print(f"Loaded {len(filtered_data[conf])} {conf} papers from 2010 onwards (out of {len(conf_data)} total)")

# Function to filter papers containing the word "attention" at least 3 times
def filter_attention_papers(data_by_conference):
    all_attention_papers = []
    
    # Process each conference separately
    for conference_name, conference_data in data_by_conference.items():
        conference_attention_papers = []
        
        for paper in tqdm(conference_data, desc=f"Filtering {conference_name} papers for 'attention'"):
            # Check if "attention" appears in the text (case insensitive) at least 3 times
            if paper["text"]:
                matches = re.findall(r'\battention\b', paper["text"], re.IGNORECASE)
                if len(matches) >= 5:  # At least 5 occurrences
                    # Extract title from text
                    title = extract_title(paper["text"])
                    
                    # Create a record with relevant information
                    paper_info = {
                        "conference": conference_name,
                        "year": paper["year"],
                        "title": title,
                        "paper_id": paper["No"],
                        "index": paper["index"],
                        "text": paper["text"],
                        "attention_count": len(matches)  # Count of "attention" occurrences
                    }
                    conference_attention_papers.append(paper_info)
        
        print(f"Found {len(conference_attention_papers)} {conference_name} papers with 'attention' mentioned at least 3 times since 2010")
        all_attention_papers.extend(conference_attention_papers)
    
    return all_attention_papers

# Helper function to extract paper title from text
def extract_title(text):
    if not text:
        return "Unknown Title"
    
    # Try to extract the first line or sentence as the title
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if line and len(line) > 10 and len(line) < 200:  # Reasonable title length
            return line
    
    # Fallback: just take the first 100 characters
    return text[:100] + "..." if len(text) > 100 else text

# Filter papers with "attention"
attention_papers = filter_attention_papers(filtered_data)

# Convert to DataFrame for easier analysis
attention_df = pd.DataFrame(attention_papers)

# Display stats by conference
print("\nAttention papers by conference:")
conference_counts = attention_df['conference'].value_counts()
print(conference_counts)

# Display stats by year
print("\nAttention papers by year:")
year_counts = attention_df['year'].value_counts().sort_index()
print(year_counts)

# Show average attention count by conference
average_attention = attention_df.groupby('conference')['attention_count'].mean()
print("\nAverage 'attention' mentions by conference:")
print(average_attention)

# Create a pivot table to analyze trends by conference and year
attention_pivot = pd.pivot_table(
    attention_df, 
    values='index', 
    index='year', 
    columns='conference', 
    aggfunc='count',
    fill_value=0
)
print("\nPivot table of attention papers by year and conference:")
print(attention_pivot)

# Store data by conference for easier access
attention_by_conference = {}
for conf in conferences:
    attention_by_conference[conf] = attention_df[attention_df['conference'] == conf]

# Identify potential breakthrough papers - early mentions of attention
early_attention_papers = {}
for conf in conferences:
    conf_df = attention_by_conference[conf]
    if not conf_df.empty:
        # Sort by year and then by attention count (descending) to find influential early papers
        early_attention_papers[conf] = conf_df[conf_df['year'] <= 2015].sort_values(['year', 'attention_count'], 
                                                                                     ascending=[True, False]).head(5)
    else:
        early_attention_papers[conf] = pd.DataFrame()  # Empty DataFrame if no papers match




Loading AI paper dataset...
Loaded 16431 NIPS papers from 2010 onwards (out of 20284 total)
Loaded 12128 CVPR papers from 2010 onwards (out of 12128 total)
Loaded 5806 ICLR papers from 2010 onwards (out of 5806 total)


Filtering NIPS papers for 'attention': 100%|██████████| 16431/16431 [00:07<00:00, 2160.62it/s]


Found 1883 NIPS papers with 'attention' mentioned at least 3 times since 2010


Filtering CVPR papers for 'attention': 100%|██████████| 12128/12128 [00:05<00:00, 2150.73it/s]


Found 3404 CVPR papers with 'attention' mentioned at least 3 times since 2010


Filtering ICLR papers for 'attention': 100%|██████████| 5806/5806 [00:04<00:00, 1359.33it/s]


Found 1221 ICLR papers with 'attention' mentioned at least 3 times since 2010

Attention papers by conference:
conference
CVPR    3404
NIPS    1883
ICLR    1221
Name: count, dtype: int64

Attention papers by year:
year
2010       2
2011       1
2012       4
2013      17
2014      14
2015      30
2016      75
2017     143
2018      97
2019     166
2020     252
2021     909
2022    1367
2023    1871
2024    1560
Name: count, dtype: int64

Average 'attention' mentions by conference:
conference
CVPR    21.348707
ICLR    28.260442
NIPS    24.936803
Name: attention_count, dtype: float64

Pivot table of attention papers by year and conference:
conference  CVPR  ICLR  NIPS
year                        
2010           0     0     2
2011           0     0     1
2012           0     0     4
2013          15     0     2
2014           9     0     5
2015          16     4    10
2016          42    12    21
2017          93    21    29
2018           0    34    63
2019           0    54   112
2020   

## Classfication by Attention Category

In [27]:
import re
import pandas as pd
import numpy as np
from collections import Counter

# Define keyword lexicons for each attention category
attention_mechanism_keywords = [
    "self-attention", "self attention", "multi-head", "multi head", 
    "attention layer", "attention module", "attention network", 
    "attention weights", "attention score", "attention vector",
    "scaled dot-product", "dot product attention", 
    "additive attention", "multiplicative attention",
    "query-key-value", "qkv", 
    "transformer", "attention is all you need",
]

attention_cognitive_keywords = [
    "cognitive attention", "selective attention", "attentional resources",
    "visual attention", "human attention", "attentional focus", 
    "attentional bias", "attention allocation",
    "cognitive load", "attentional control", "human-like attention",
    "biological attention", "neuroscience", "cognitive science", "psychology of attention",
    "attentional process", "attention model", "attention theory", 
    "perceptual attention", "attentional spotlight", "focus of attention"
]

attention_explanatory_keywords = [
    "attention visualization", "attention map", "attention heatmap", 
    "visualize attention", "attention rollout", "attention flow",
    "interpretability", "explainability", "explain", "visualize", 
    "attention interpretation", "attention analysis", "attention pattern",
    "attention behavior", "attention activation", "attention highlighting",
    "attribution", "saliency", "feature importance", "what model attends to",
    "attention-based explanation", "attention for understanding"
]

# Patterns for general language usage of "attention"
general_attention_patterns = [
    r"receives?\s+attention",
    r"pays?\s+attention",
    r"draws?\s+attention",
    r"attention\s+of\s+researchers",
    r"attention\s+in\s+the\s+literature",
    r"has\s+received\s+attention",
    r"give\s+attention\s+to",
    r"gave\s+attention\s+to",
    r"giving\s+attention\s+to",
    r"brings?\s+attention\s+to",
    r"brought\s+attention\s+to",
    r"bringing\s+attention\s+to",
    r"focus\s+attention\s+on",
    r"focused\s+attention\s+on",
    r"call\s+attention\s+to",
    r"calls\s+attention\s+to",
    r"called\s+attention\s+to",
    r"devote\s+attention\s+to",
    r"devoted\s+attention\s+to",
    r"directing\s+attention\s+to",
    r"direct\s+attention\s+to",
    r"directed\s+attention\s+to",
    r"attract\s+attention",
    r"attracts\s+attention",
    r"attracted\s+attention",
    r"get\s+attention",
    r"gets\s+attention",
    r"got\s+attention",
    r"attention\s+to\s+detail",
    r"attention\s+span",
    r"lack\s+of\s+attention",
    r"short\s+attention",
    r"limited\s+attention",
    r"insufficient\s+attention",
    r"enough\s+attention"
]

def classify_attention_paper(text, title=""):
    """
    Classify a paper based on which sense of attention it predominantly uses.
    Returns a dictionary with scores for each category.
    """
    if not text:
        return {
            "mechanism": 0,
            "cognitive": 0,
            "explanatory": 0,
            "primary_category": "unknown",
            "confidence": 0,
            "mechanism_count": 0,
            "cognitive_count": 0,
            "explanatory_count": 0,
            "general_usage_count": 0,
            "total_attention_mentions": 0
        }
    
    # Combine title and text, giving more weight to title
    full_text = (title + " " + title + " " + text).lower()
    
    # Count matches for each category
    mechanism_count = sum(full_text.count(keyword.lower()) for keyword in attention_mechanism_keywords)
    cognitive_count = sum(full_text.count(keyword.lower()) for keyword in attention_cognitive_keywords)
    explanatory_count = sum(full_text.count(keyword.lower()) for keyword in attention_explanatory_keywords)
    
    # Count general language usage of "attention"
    general_usage_count = sum(len(re.findall(pattern, full_text)) for pattern in general_attention_patterns)
    
    # Count total mentions of "attention"
    total_attention_mentions = len(re.findall(r'\battention\b', full_text))
    
    # Calculate technical mentions (mechanism + cognitive + explanatory)
    # This is a rough approximation
    technical_mentions = mechanism_count + cognitive_count + explanatory_count
    
    # If we have only general usage or very few technical matches compared to general usage
    if technical_mentions == 0 or (general_usage_count > 0 and technical_mentions / general_usage_count < 0.5):
        return {
            "mechanism": 0,
            "cognitive": 0,
            "explanatory": 0,
            "primary_category": "general_usage",
            "confidence": 1.0,
            "mechanism_count": mechanism_count,
            "cognitive_count": cognitive_count,
            "explanatory_count": explanatory_count,
            "general_usage_count": general_usage_count,
            "total_attention_mentions": total_attention_mentions
        }
    
    # Calculate total relevant technical mentions
    total_mentions = mechanism_count + cognitive_count + explanatory_count
    
    if total_mentions == 0:
        return {
            "mechanism": 0,
            "cognitive": 0,
            "explanatory": 0,
            "primary_category": "unknown",
            "confidence": 0,
            "mechanism_count": 0,
            "cognitive_count": 0,
            "explanatory_count": 0,
            "general_usage_count": general_usage_count,
            "total_attention_mentions": total_attention_mentions
        }
    
    # Calculate proportions
    mechanism_score = mechanism_count / total_mentions
    cognitive_score = cognitive_count / total_mentions
    explanatory_score = explanatory_count / total_mentions
    
    # Determine primary category
    scores = {
        "mechanism": mechanism_score,
        "cognitive": cognitive_score,
        "explanatory": explanatory_score
    }
    primary_category = max(scores, key=scores.get)
    
    # Calculate confidence (difference between highest and second highest score)
    sorted_scores = sorted(scores.values(), reverse=True)
    if len(sorted_scores) > 1:
        confidence = sorted_scores[0] - sorted_scores[1]
    else:
        confidence = 1.0
    
    return {
        "mechanism": mechanism_score,
        "cognitive": cognitive_score,
        "explanatory": explanatory_score,
        "primary_category": primary_category,
        "confidence": confidence,
        "mechanism_count": mechanism_count,
        "cognitive_count": cognitive_count,
        "explanatory_count": explanatory_count,
        "general_usage_count": general_usage_count,
        "total_attention_mentions": total_attention_mentions
    }

def extract_context_window(text, keyword="attention", window_size=100):
    """
    Extract context windows around keyword mentions
    """
    if not text:
        return []
    
    text = text.lower()
    keyword = keyword.lower()
    contexts = []
    
    # Find all positions of the keyword
    start_positions = [m.start() for m in re.finditer(r'\b' + re.escape(keyword) + r'\b', text)]
    
    for pos in start_positions:
        # Calculate window boundaries
        start = max(0, pos - window_size)
        end = min(len(text), pos + len(keyword) + window_size)
        
        # Extract context
        context = text[start:end]
        
        # Check if this is likely a general usage or technical usage
        if not any(re.search(pattern, context) for pattern in general_attention_patterns):
            contexts.append({
                "text": context,
                "is_general_usage": False
            })
        else:
            contexts.append({
                "text": context,
                "is_general_usage": True
            })
    
    return contexts

def analyze_attention_papers(papers_df):
    """
    Analyze and classify papers in a DataFrame
    """
    # Print column names to help with debugging
    print("Columns in DataFrame:", papers_df.columns.tolist())
    
    # Create columns for classification results
    results = []
    
    for _, paper in papers_df.iterrows():
        try:
            # Get the paper text and title
            paper_text = paper.get("text", "")
            paper_title = paper.get("title", "")
            
            # Classify the paper
            classification = classify_attention_paper(paper_text, paper_title)
            
            # Extract context windows for "attention" mentions
            contexts = extract_context_window(paper_text)
            
            # Filter out general usage contexts
            technical_contexts = [ctx["text"] for ctx in contexts if not ctx["is_general_usage"]]
            general_contexts = [ctx["text"] for ctx in contexts if ctx["is_general_usage"]]
            
            # Sample contexts
            sample_technical_context = technical_contexts[0] if technical_contexts else ""
            sample_general_context = general_contexts[0] if general_contexts else ""
            
            # Create a result dictionary with safe access
            result = {
                "conference": paper.get("conference", "unknown"),
                "year": paper.get("year", 0),
                "title": paper_title,
                "text": paper_text,
                "primary_category": classification.get("primary_category", "unknown"),
                "confidence": classification.get("confidence", 0),
                "mechanism_score": classification.get("mechanism", 0),
                "cognitive_score": classification.get("cognitive", 0),
                "explanatory_score": classification.get("explanatory", 0),
                "mechanism_count": classification.get("mechanism_count", 0),
                "cognitive_count": classification.get("cognitive_count", 0),
                "explanatory_count": classification.get("explanatory_count", 0),
                "general_usage_count": classification.get("general_usage_count", 0),
                "total_attention_mentions": classification.get("total_attention_mentions", 0),
                "technical_context_count": len(technical_contexts),
                "general_context_count": len(general_contexts),
                "sample_technical_context": sample_technical_context,
                "sample_general_context": sample_general_context
            }
            results.append(result)
        except Exception as e:
            print(f"Error processing paper: {e}")
            # Add a minimal record to keep track of errors
            results.append({
                "conference": paper.get("conference", "unknown"),
                "year": paper.get("year", 0),
                "title": paper.get("title", "Error"),
                "primary_category": "error",
                "confidence": 0,
                "mechanism_score": 0,
                "cognitive_score": 0,
                "explanatory_score": 0,
                "mechanism_count": 0,
                "cognitive_count": 0,
                "explanatory_count": 0,
                "general_usage_count": 0,
                "total_attention_mentions": 0,
                "technical_context_count": 0,
                "general_context_count": 0,
                "sample_technical_context": "",
                "sample_general_context": f"Error: {str(e)}"
            })
    
    # Create a new DataFrame with results
    results_df = pd.DataFrame(results)
    
    return results_df

# Analyze our attention papers
try:
    print("Analyzing papers for different senses of 'attention'...")
    attention_classification = analyze_attention_papers(attention_df)
    
    # Summary statistics
    print("\nPaper Classification Summary:")
    print(attention_classification["primary_category"].value_counts())
    
    # Calculate percentage of papers with general vs technical usage
    total_papers = len(attention_classification)
    general_usage_papers = len(attention_classification[attention_classification["primary_category"] == "general_usage"])
    technical_usage_papers = total_papers - general_usage_papers
    
    print(f"\nGeneral vs Technical Usage:")
    print(f"General language usage: {general_usage_papers} papers ({general_usage_papers/total_papers*100:.1f}%)")
    print(f"Technical AI usage: {technical_usage_papers} papers ({technical_usage_papers/total_papers*100:.1f}%)")
    
    # Show distribution by conference (excluding general usage)
    technical_papers = attention_classification[attention_classification["primary_category"] != "general_usage"]
    if not technical_papers.empty:
        conference_category_pivot = pd.pivot_table(
            technical_papers, 
            index="conference", 
            columns="primary_category", 
            aggfunc="size", 
            fill_value=0
        )
        print("\nDistribution of technical usage by conference:")
        print(conference_category_pivot)
    
    # Show distribution by year (evolution over time)
    if not technical_papers.empty:
        year_category_pivot = pd.pivot_table(
            technical_papers, 
            index="year", 
            columns="primary_category", 
            aggfunc="size", 
            fill_value=0
        )
        print("\nEvolution of technical usage over time:")
        print(year_category_pivot)
    
    # Show average confidence by category
    category_confidence = attention_classification.groupby("primary_category")["confidence"].mean()
    print("\nAverage confidence by category:")
    print(category_confidence)
    
    
    
except Exception as e:
    print(f"Error during analysis: {e}")
    import traceback
    traceback.print_exc()

Analyzing papers for different senses of 'attention'...
Columns in DataFrame: ['conference', 'year', 'title', 'paper_id', 'index', 'text', 'attention_count']

Paper Classification Summary:
primary_category
mechanism        5285
explanatory       925
cognitive         243
general_usage      55
Name: count, dtype: int64

General vs Technical Usage:
General language usage: 55 papers (0.8%)
Technical AI usage: 6453 papers (99.2%)

Distribution of technical usage by conference:
primary_category  cognitive  explanatory  mechanism
conference                                         
CVPR                     97          472       2812
ICLR                     42          176        992
NIPS                    104          277       1481

Evolution of technical usage over time:
primary_category  cognitive  explanatory  mechanism
year                                               
2010                      1            1          0
2011                      1            0          0
2012         

## Classfication by Research Domain

In [28]:
import pandas as pd
import re
from collections import Counter

# Define clear domain identifiers - direct and specific terms that strongly indicate the domain
domain_identifiers = {
    "LLM": ["large language model", "llm", "gpt", "language model", "transformer",
           "generative pre-trained"],
    
    "CV": ["computer vision", "image recognition", "object detection", "image classification", 
          "image segmentation", "convolutional neural network", "cnn", "vision transformer"],
    
    "NLP": ["natural language processing", "nlp", "text classification", "word embedding"],
    
    "RL": ["reinforcement learning", "rl", "q-learning", "policy gradient"]
}

def classify_ai_domain(text, title=""):
    """
    Classify a paper into LLM, CV, NLP, or RL domains using only direct identifiers.
    If no direct identifiers are found, classify as "unknown".
    """
    if not text:
        return {
            "domain": "unknown",
            "confidence": 0,
            "keywords_found": []
        }
    
    # Combine title and text, with title weighted more
    full_text = (title + " " + title + " " + text).lower()
    
    # Check for direct domain identifiers
    domain_matches = {}
    domain_terms_found = {}
    
    for domain, identifiers in domain_identifiers.items():
        terms_found = []
        count = 0
        for term in identifiers:
            occurrences = full_text.count(term.lower())
            if occurrences > 0:
                count += occurrences
                terms_found.append(term)
        
        domain_matches[domain] = count
        domain_terms_found[domain] = terms_found
    
    # If we have direct identifier matches, use those to determine domain
    if any(count > 0 for count in domain_matches.values()):
        # Find domain with most matches
        best_domain = max(domain_matches, key=domain_matches.get)
        
        return {
            "domain": best_domain,
            "match_count": domain_matches[best_domain],
            "confidence": 1.0,  # High confidence since we matched direct identifiers
            "keywords_found": domain_terms_found[best_domain]
        }
    
    # If no direct identifiers found, return unknown
    return {
        "domain": "unknown",
        "match_count": 0,
        "confidence": 0,
        "keywords_found": []
    }

def analyze_domains(papers_df):
    """
    Analyze and classify papers by AI domain using only direct identifiers
    """
    print("Classifying papers by AI domain (LLM, CV, NLP, RL)...")
    print("Note: Only classifying papers with direct domain identifiers.")
    
    # Create columns for classification results
    results = []
    
    for _, paper in papers_df.iterrows():
        try:
            # Get the paper text and title
            paper_text = paper.get("text", "")
            paper_title = paper.get("title", "")
            
            # Classify the paper by AI domain
            domain_classification = classify_ai_domain(paper_text, paper_title)
            
            # Get attention category if available
            attention_category = paper.get("primary_category", "unknown")
            
            # Create a result dictionary
            result = {
                "conference": paper.get("conference", "unknown"),
                "year": paper.get("year", 0),
                "title": paper_title,
                "domain": domain_classification.get("domain", "unknown"),
                "match_count": domain_classification.get("match_count", 0),
                "keywords": ", ".join(domain_classification.get("keywords_found", [])[:5]),
                "attention_category": attention_category
            }
            results.append(result)
        except Exception as e:
            print(f"Error processing paper: {e}")
            # Add a minimal record to keep track of errors
            results.append({
                "conference": paper.get("conference", "unknown"),
                "year": paper.get("year", 0),
                "title": paper.get("title", "Error"),
                "domain": "error",
                "match_count": 0,
                "keywords": "",
                "attention_category": paper.get("primary_category", "unknown")
            })
    
    # Create a new DataFrame with results
    results_df = pd.DataFrame(results)
    
    return results_df

# Main analysis function that runs on the attention classification data
def run_domain_analysis(attention_classification_df):
    """
    Run the domain analysis on the previously classified attention papers
    """
    try:
        # Classify papers by AI domain
        domain_classification = analyze_domains(attention_classification_df)
        
        # Summary statistics
        print("\nDomain Classification Summary:")
        print(domain_classification["domain"].value_counts())
        
        # Calculate percentage of classified papers
        total_papers = len(domain_classification)
        classified_papers = len(domain_classification[domain_classification["domain"] != "unknown"])
        classification_rate = classified_papers / total_papers * 100 if total_papers > 0 else 0
        
        print(f"\nClassification rate: {classification_rate:.1f}% ({classified_papers}/{total_papers} papers)")
        
        # Show distribution by conference
        conference_domain_pivot = pd.pivot_table(
            domain_classification, 
            index="conference", 
            columns="domain", 
            aggfunc="size", 
            fill_value=0
        )
        print("\nDistribution by conference:")
        print(conference_domain_pivot)
        
        # Show distribution by year (evolution over time)
        year_domain_pivot = pd.pivot_table(
            domain_classification, 
            index="year", 
            columns="domain", 
            aggfunc="size", 
            fill_value=0
        )
        print("\nEvolution over time:")
        print(year_domain_pivot)
        
        # Cross-tabulate domain vs attention category
        if "attention_category" in domain_classification.columns:
            domain_attention_pivot = pd.pivot_table(
                domain_classification,
                index="domain",
                columns="attention_category",
                aggfunc="size",
                fill_value=0
            )
            print("\nDomain vs Attention Category:")
            print(domain_attention_pivot)
        
        return domain_classification, conference_domain_pivot, year_domain_pivot, domain_examples
        
    except Exception as e:
        print(f"Error during domain analysis: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None, None


try:
    # Check if we have the attention classification data from the previous step
    if 'attention_classification' in globals():
        print("Running domain analysis on previously classified attention papers...")
        domain_classification, conference_domain_pivot, year_domain_pivot, domain_examples = run_domain_analysis(attention_classification)
        
    else:
        print("Error: attention_classification not found. Please run the attention classification step first.")
except Exception as e:
    print(f"Error in main execution: {e}")
    import traceback
    traceback.print_exc()

Running domain analysis on previously classified attention papers...
Classifying papers by AI domain (LLM, CV, NLP, RL)...
Note: Only classifying papers with direct domain identifiers.

Domain Classification Summary:
domain
CV     2799
LLM    1905
RL     1743
NLP      61
Name: count, dtype: int64

Classification rate: 100.0% (6508/6508 papers)

Distribution by conference:
domain        CV  LLM  NLP   RL
conference                     
CVPR        2046  816   10  532
ICLR         217  456   22  526
NIPS         536  633   29  685

Evolution over time:
domain   CV  LLM  NLP   RL
year                      
2010      1    0    0    1
2011      0    0    0    1
2012      0    1    0    3
2013     11    0    0    6
2014      6    0    0    8
2015     11    2    0   17
2016     41    3    3   28
2017     90    4    9   40
2018     26   11    4   56
2019     61   30    6   69
2020     66   61    5  120
2021    414  220   14  261
2022    600  435    7  325
2023    798  598   11  464
2024    674